In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
image_path = "test.png"
image_bgr = cv2.imread(image_path)

In [4]:
mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)

DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

In [5]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, enable_segmentation=True) as holistic:
    # Recolor Feed
    image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    # Make Detections
    results = holistic.process(image)
    # print(results.face_landmarks)

    # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

    # Recolor image back to BGR for rendering
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 1. Draw face landmarksq
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )

    # 2. Right hand
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )

    # 3. Left Hand
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )

    # 4. Pose Detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

    cv2.imshow('Raw Webcam Feed', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
annotated_image = image.copy()
condition = np.stack((results.segmentation_mask,) * 3, axis = -1) >0.1
bg_image = image.copy()
bg_image = cv2.blur(bg_image, (5, 5))
annotated_image = np.where(condition, annotated_image, bg_image)
cv2.imshow('Mask', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, enable_segmentation=True) as holistic:
    # Recolor Feed
    image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    # Make Detections
    results = holistic.process(image)
    # print(results.face_landmarks)

    # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

    # Recolor image back to BGR for rendering
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    annotated_image = image.copy()
    condition = np.stack((results.segmentation_mask,) * 3, axis = -1) >0.1
    bg_image = image.copy()
    bg_image = cv2.blur(bg_image, (20, 20))
    annotated_image = np.where(condition, annotated_image, bg_image)
    vis = np.concatenate((image, annotated_image), axis = 1)
    cv2.imshow('image', vis)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
def finger_check(hand_landmarks):
    thumbIsOpen = 0
    firstFingerIsOpen = 0
    secondFingerIsOpen = 0
    thirdFingerIsOpen = 0
    fourthFingerIsOpen = 0

    if (hand_landmarks[3].x < hand_landmarks[2].x and hand_landmarks[4].x < hand_landmarks[2].x):
        thumbIsOpen = 1
    if (hand_landmarks[7].y < hand_landmarks[6].y and hand_landmarks[8].y < hand_landmarks[6].y):
        firstFingerIsOpen = 1
    if (hand_landmarks[11].y < hand_landmarks[10].y and hand_landmarks[12].y < hand_landmarks[10].y):
        secondFingerIsOpen = 1
    if (hand_landmarks[15].y < hand_landmarks[14].y and hand_landmarks[16].y < hand_landmarks[14].y):
        thirdFingerIsOpen = 1
    if (hand_landmarks[19].y < hand_landmarks[18].y and hand_landmarks[20].y < hand_landmarks[18].y):
        fourthFingerIsOpen = 1
    return [thumbIsOpen, firstFingerIsOpen, secondFingerIsOpen, thirdFingerIsOpen, fourthFingerIsOpen]

In [22]:
cap = cv2.VideoCapture(0)
function_select = 0
text = 'Mode not checked'
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, enable_segmentation=True) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        annotated_image = image.copy()
        
        if (results.left_hand_landmarks):
            finger_state = finger_check(results.left_hand_landmarks.landmark)

            if finger_state[0] == 0 and finger_state[1] == 0 and finger_state[2] == 0 and finger_state[3] == 0 and finger_state[4] == 0:
                text = 'fist'
                function_select = 0
                
            if finger_state[0] == 1 and finger_state[1] == 1 and finger_state[2] == 1 and finger_state[3] == 1 and finger_state[4] == 1:
                text = 'Opened'
                function_select = 1

            if finger_state[2] == 1 and finger_state[1] == 0 and finger_state[3] == 0:
                text = 'middle finger'
                function_select = 2
        
        if function_select == 1:
            condition = np.stack((results.segmentation_mask,) * 3, axis = -1) >0.1
            bg_image = image.copy()
            bg_image = cv2.blur(bg_image, (10, 10))
            annotated_image = np.where(condition, annotated_image, bg_image)
        
        elif function_select == 2:
            condition = np.stack((results.segmentation_mask,) * 3, axis = -1) > 0.1
            annotated_image = cv2.blur(annotated_image, (30, 30))
            bg_image = image.copy()
            annotated_image = np.where(condition, annotated_image, bg_image)
        
        else:
            annotated_image = image.copy()
        
        vis = np.concatenate((image, annotated_image), axis = 1)
        cv2.putText(vis, text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)
        cv2.imshow('Raw Webcam Feed', vis)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [10]:
finger_state = finger_check(results.left_hand_landmarks.landmark)
print(finger_state)

AttributeError: 'NoneType' object has no attribute 'landmark'